Import Libraries / Packages

In [24]:
from PyQt5.QtCore import *
import pandas as pd
from qgis.core import *
import os
import geopandas as gpd
import fiona
import sys
import math
import numpy
from shapely.geometry import shape
from shapely.geometry import Polygon
from rtree import index


In [2]:
# Increase width of notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Define functions to be used in script

In [3]:
# Create function to check for transect numbers of before and after points

def transectCheck(rownumber, dataframe):
    current = dataframe['Transect'].iloc[rownumber]
    after = dataframe['Transect'].iloc[rownumber + 1]
    if "/" in current:
        slash1 = current.find("/")
        slash2 = slash1 + 1
        current = current[slash2:]
    else: pass
    if "/" in after:
        slash = after.find("/")
        after = after[:slash]

    else: pass
    
    if current == after:
        return 1
    else: return 0
    
    
# Create function to determine if a point is a mapping unit

def muCheck(rownumber, dataframe):
    # Check transects
    currentT = dataframe['Transect'].iloc[rownumber]
    currentT2 = dataframe['Transect'].iloc[rownumber]    
    beforeT = dataframe['Transect'].iloc[rownumber - 1]
    afterT = dataframe['Transect'].iloc[rownumber + 1]
    if "/" in currentT:
        slash1 = currentT.find("/")
        slash2 = slash1 + 1
        currentT = currentT[slash2:]
        currentT2 = currentT2[:slash1]
    else: pass
    if "/" in afterT:
        slash = afterT.find("/")
        afterT = afterT[:slash]
    else: pass
    if "/" in beforeT:
        slash3 = beforeT.find("/")
        slash4 = slash3 + 1
        beforeT = beforeT[slash4:]
    else: pass
    
    # Check subclasses
    currentS = dataframe['Subclass'].iloc[rownumber][:2]
    beforeS = dataframe['Subclass'].iloc[rownumber - 1][:2]
    afterS = dataframe['Subclass'].iloc[rownumber + 1][:2]

    if currentT == afterT and currentS == afterS: pass
    else:
            if currentT2 == beforeT and currentS == beforeS: pass
            else: return 2
            

# Create function to determine if a point is a mapping unit for data with no transect field
            
def muCheckNoT(rownumber, dataframe):    
    # Check subclasses
    currentS = dataframe['Subclass'].iloc[rownumber][:2]
    beforeS = dataframe['Subclass'].iloc[rownumber - 1][:2]
    afterS = dataframe['Subclass'].iloc[rownumber + 1][:2]
    
    if currentS == afterS: pass
    else:
        if currentS == beforeS: pass
        else: return 2
            
def muCheckLast(rownumber, dataframe):
    # Check final subclass with [-1] subclass only
    currentS = dataframe['Subclass'].iloc[rownumber][:2]
    beforeS = dataframe['Subclass'].iloc[rownumber - 1][:2]
    
    if currentS == beforeS: pass
    else: return 2
    
    
# Define function to test whether angle of mu corresponds to pre or post angle

def prepostCheck(rownumber, dataframe):
    currentT = dataframe['Transect'].iloc[rownumber]
    afterT = dataframe['Transect'].iloc[rownumber + 1]
    
    if "/" in currentT:
        slash1 = currentT.find("/")
        slash2 = slash1 + 1
        currentT = currentT[slash2:]
    else: pass
    
    if "/" in afterT:
        slash = afterT.find("/")
        afterT = afterT[:slash]
    else: pass
    
    if currentT == afterT: return "post"
    else: return "pre"
    
    
def prepostCheckNoT(rownumber, dataframe):
    # Check subclasses
    currentS = dataframe['Subclass'].iloc[rownumber][:2]
    beforeS = dataframe['Subclass'].iloc[rownumber - 1][:2]
    afterS = dataframe['Subclass'].iloc[rownumber + 1][:2]
    
    if currentS == afterS: return "post:"
    else: return "pre"

Set Up the Data - Specify correct folders to be generated

In [27]:
# Create directories within selected folder
root = input("Enter file path to site folder in EPA Salt Marsh UAS Study with quotations -- ")
root = root[1:-1]
base = root + "\Jupyter_Working_Folder"
XY_Points_Folder = base + "\XY_Points"
Lines_Folder = base + "\Lines"
Buffer_Folder = base + "\Lines_Buffered"
Class_Lines_Folder = base + "\Class_Lines"
Class_Buffer_Folder = base + "\Class_Lines_Buffered"
mu_Folder = base + "\mu"
mu_Buffer_Folder = base + "\mu_Buffered"
working = base + "\working"
Polygon_Folder = base + "\Polygons"
folderList = [XY_Points_Folder, Lines_Folder, Buffer_Folder, mu_Folder, mu_Buffer_Folder, 
              Class_Lines_Folder, Class_Buffer_Folder, working, Polygon_Folder]
for i in folderList:
    if not os.path.exists(i):
        os.makedirs(i)

Enter file path to site folder in EPA Salt Marsh UAS Study with quotations -- "G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Newbury"


Check data to edit cell below to determine number of rows to exclude

In [5]:
# Read in .cvs with Transects
# Keep quotations around file path in the input
read_data = input("Enter file path to ground truthing data .csv here -- ")
read_data = read_data[1:-1]

# skiprows argument may need to be tweaked depending on format of the .csv's headers
df = pd.read_csv(read_data, skiprows=1)
# Observe number of rows to discard including RTC data
df.head()

Enter file path to ground truthing data .csv here -- "G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Newbury\Ground Truthing\NAD83_UTM19N\oldtownhill14oct2018.csv"


,RTCM0042,Unnamed: 1,4747306.228,345583.160,16.646
0,1,22r,4737159.533,347668.963,0.695
1,2,22r,4737157.980,347669.880,0.726
2,3,22r,4737157.053,347671.442,0.785
3,4,22r,4737155.156,347673.739,0.976
4,5,22r,4737151.725,347671.306,0.342


In [6]:
# # Delete top row with projection/datum and RTC data
# datos = df.iloc[1:]
# # Creating data frame
dato = pd.DataFrame(df)

In [7]:
# Rename column headers
dato.columns = ['Point Number', 'Subclass', 'Northing', 'Easting', 'Altitude']

In [8]:
# # Run this section if dealing with transect field
# # Filter for null values where Transect != NaN 
# filter_dato = dato[dato['Transect'].notnull()]

In [9]:
# Run this cell if you did not run the previous cell
# Assign to filter_dato to not mess with remaining code
filter_dato = dato

In [10]:
# Create empty fields for transect angles, Class, and mapping unit

filter_dato['Pre Angle'] = ""
filter_dato['Post Angle'] = ""
filter_dato['Class'] = ""
filter_dato['mu'] = ""

# Preview the data
filter_dato.head()

,Point Number,Subclass,Northing,Easting,Altitude,Pre Angle,Post Angle,Class,mu
0,1,22r,4737159.533,347668.963,0.695,,,,
1,2,22r,4737157.980,347669.880,0.726,,,,
2,3,22r,4737157.053,347671.442,0.785,,,,
3,4,22r,4737155.156,347673.739,0.976,,,,
4,5,22r,4737151.725,347671.306,0.342,,,,


In [11]:
# Define variables of lists of coordinates of current, before, and after coordinates
# This will be used to calculate angle from one point to the next

y = 0
precoordN = []
for x in filter_dato.iterrows():
    precoordN.append(filter_dato['Northing'].iloc[y-1])
    y += 1
y = 0
precoordE = []
for x in filter_dato.iterrows():
    precoordE.append(filter_dato['Easting'].iloc[y-1])
    y += 1
y = 0
currcoordN = []
for x in filter_dato.iterrows():
    currcoordN.append(filter_dato['Northing'].iloc[y])
    y += 1
y = 0
currcoordE = []
for x in filter_dato.iterrows():
    currcoordE.append(filter_dato['Easting'].iloc[y])
    y += 1
y = 0
postcoordN = []
for x in filter_dato.iterrows():
    if y+1 < len(filter_dato):
        postcoordN.append(filter_dato['Northing'].iloc[y+1])
        y += 1
    else: break
y = 0
postcoordE = []
for x in filter_dato.iterrows():
    if y+1 < len(filter_dato):
        postcoordE.append(filter_dato['Easting'].iloc[y+1])
        y += 1
    else: break

In [12]:
# Append calculated angles to lists
y = 0
preAngle = []
for i in filter_dato.iterrows():
    if y-1 < 0:
        preAngle.append(0)
        y += 1
    else:
        myradians = math.atan2(currcoordN[y]-precoordN[y], currcoordE[y]-precoordE[y])
        mydegrees = math.degrees(myradians)
        preAngle.append(mydegrees)
        y += 1

y = 0
postAngle = []
for i in filter_dato.iterrows():
    if y+1 < len(filter_dato):
        myradians = math.atan2(postcoordN[y]-currcoordN[y], postcoordE[y]-currcoordE[y])
        mydegrees = math.degrees(myradians)
        postAngle.append(mydegrees)
        y += 1
    else: 
        postAngle.append(0)
        y += 1       

In [13]:
# Parse out Class from Subclass and append to new list

classe = []
y = 0
for i in filter_dato.iterrows():
    if filter_dato['Subclass'].iloc[y][:1] == "0" or filter_dato['Subclass'].iloc[y][:1] == "1":
        classe.append(1)
        y += 1
    elif filter_dato['Subclass'].iloc[y][:1] == "2":
        classe.append(2)
        y += 1
    elif filter_dato['Subclass'].iloc[y][:1] == "3":
        classe.append(3)
        y += 1

In [14]:
# Add lists to empty columns

filter_dato['Pre Angle'] = preAngle
filter_dato['Post Angle'] = postAngle
filter_dato['Class'] = classe

In [15]:
# Create new shapefile for Points that do not create a line
y = 0
muList = []

# for loop to write point for every mapping unit
for feat in filter_dato.iterrows():
    # Create if statement so you y is never > the length of your data set; this will end your script
    if y + 1 < len(filter_dato):
        if muCheckNoT(y, filter_dato) == 2:
            muList.append(1)
            
            

            y += 1
        else: 
            muList.append(0)
            y += 1
    else:
        if muCheckLast(y, filter_dato) == 2: muList.append(1)
        else: muList.append(0)

In [16]:
# Append muList to dataframe
filter_dato['mu'] = muList

In [17]:
# Create new column in dataframe to signify if the mu uses the pre or post angle
filter_dato["Pre/Post"] = ""

In [18]:
# Create for loop to create list of pre or post to append to dataframe

y = 0
prepost = []

for i in filter_dato.iterrows():
    if y + 1 < len(filter_dato):
        pp = prepostCheckNoT(y, filter_dato)
        prepost.append(pp)
        y += 1
    else: prepost.append("pre")

In [19]:
# Append prepost list to dataframe

filter_dato["Pre/Post"] = prepost

filter_dato2 = filter_dato

In [20]:
waterFilter = filter_dato2.loc[filter_dato2['Class'] == 2]
filter_dato = filter_dato.loc[filter_dato['Class'] != 2]

In [136]:
# Create QGIS XY Point Data into a List based on coordinates
point_list = [QgsPointXY(r['Easting'], r['Northing']) for i, r in filter_dato.iterrows() ]

In [137]:
# Create Multipoint list
points = QgsGeometry.fromMultiPointXY(point_list)

Create the XY Point Shapefile

In [138]:
# Create fields with correct data types

layerFields = QgsFields()
layerFields.append(QgsField('PointNum', QVariant.Int))
layerFields.append(QgsField('SubClass', QVariant.String))
layerFields.append(QgsField('Northing', QVariant.Double))
layerFields.append(QgsField('Easting', QVariant.Double))
layerFields.append(QgsField('Altitude', QVariant.Double))
layerFields.append(QgsField('Pre Angle', QVariant.Double))
layerFields.append(QgsField('Post Angle', QVariant.Double))
layerFields.append(QgsField('Class', QVariant.Int))
layerFields.append(QgsField('mu', QVariant.Int))
layerFields.append(QgsField('Pre/Post', QVariant.String))

True

In [139]:
# Write XY Point Data for each row in .csv

# Specify output to variable
XY_Output_Path = XY_Points_Folder

In [140]:
# Use for loop to create shapefile
y = 0
for x in filter_dato.itertuples():
    # Specify file output and name of .shp
    file = XY_Output_Path + "\XYPoint" + str(y) + ".shp"
    # Set type (QgsWkbTypes.), CRS (EPSG:26919), and type of file (ESRI Shapefile)
    writer = QgsVectorFileWriter(file, 'UTF-8', layerFields, QgsWkbTypes.MultiPoint, QgsCoordinateReferenceSystem('EPSG:26919'), 'ESRI Shapefile')
    # Appends X and Y coordinates to variable 'point1' at the y position (starts at 0 and adds 1 for each iteration of the for loop)
    point1 = QgsGeometry.fromPointXY(point_list[y])
    # Create an empty feature
    feat = QgsFeature()
    # Set feature's geometry to that of point1
    feat.setGeometry(point1)
    # Set values from dataframe to attributes
    # Values will be applied in the order you created the layerFields
    feat.setAttributes([int(filter_dato['Point Number'].iloc[y]), \
                        filter_dato['Subclass'].iloc[y], filter_dato['Northing'].iloc[y].item(), \
                        filter_dato['Easting'].iloc[y].item(), filter_dato['Altitude'].iloc[y].item(), \
                        filter_dato['Pre Angle'].iloc[y].item(), \
                        filter_dato['Post Angle'].iloc[y].item(), filter_dato['Class'].iloc[y].item(), \
                        filter_dato['mu'].iloc[y].item(), filter_dato['Pre/Post'].iloc[y]])
    # Add 1 to y to continue through the dataframe during next for loop
    y += 1
    # write the feature
    writer.addFeature(feat)

del(writer)

C:\Users\brett\anaconda3\envs\CAFE\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: QgsVectorFileWriter constructor is deprecated
  import sys


Concatenate all XY Point shapefiles into a single shapefile

In [141]:
# Specify which folder to look in
file = os.listdir(XY_Output_Path)

# Look for all files ending in ".shp"
path = [os.path.join(XY_Output_Path, i) for i in file if ".shp" in i]

# Concatenate to file specified in last line of code
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
# Write to same folder + name of new folder to be created which will include the shapefile of compiled XY Points
XY_Points_Comp = "\XY_Points_Comp"

In [142]:
# Write concatenated files to output path
gdf.to_file(XY_Output_Path + XY_Points_Comp )

In [143]:
# Sort shapefile by Point Number attribute
# Assign XY_Points__Outfile to where the sorted compiled XY Points will be saved
XY_Points_outfile = XY_Output_Path + XY_Points_Comp + "\\XY_Points_Comp_Sort.shp"
# Designate which compiled shapefile will be sorted
filePath = XY_Output_Path + XY_Points_Comp + XY_Points_Comp + ".shp"
# Use geopandas to read in shapefile - Append read shapefile to variable 'shape'
shape = gpd.read_file(filePath)
# Sort based on Point Number
shape_sort = shape.iloc[shape['PointNum'].sort_values().index.values]

In [144]:
# Re-write file back out as sorted compiled shapefile
shape_sort.to_file(driver = 'ESRI Shapefile', filename = XY_Points_outfile)

In [145]:
XY_Points_outfile

'G:\\.shortcut-targets-by-id\\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\\SaltMUAS_share\\EPA Salt Marsh UAS Study\\Newbury\\Jupyter_Working_Folder\\XY_Points\\XY_Points_Comp\\XY_Points_Comp_Sort.shp'

Create PolyLine Shapefile

In [146]:
# Specify empty folder to put the Line files in
linesFolder = Lines_Folder

In [147]:
# # Create function to determine if a point is a mapping unit

# def muCheck(rownumber, dataframe):
#     # Check transects
#     currentT = dataframe['Transect'].iloc[rownumber]
#     currentT2 = dataframe['Transect'].iloc[rownumber]    
#     beforeT = dataframe['Transect'].iloc[rownumber - 1]
#     afterT = dataframe['Transect'].iloc[rownumber + 1]
#     if "/" in currentT:
#         slash1 = currentT.find("/")
#         slash2 = slash1 + 1
#         currentT = currentT[slash2:]
#         currentT2 = currentT2[:slash1]
#     else: pass
#     if "/" in afterT:
#         slash = afterT.find("/")
#         afterT = afterT[:slash]
#     else: pass
#     if "/" in beforeT:
#         slash3 = beforeT.find("/")
#         slash4 = slash3 + 1
#         beforeT = beforeT[slash4:]
#     else: pass
    
#     # Check subclasses
#     currentS = dataframe['Subclass'].iloc[rownumber][:2]
#     beforeS = dataframe['Subclass'].iloc[rownumber - 1][:2]
#     afterS = dataframe['Subclass'].iloc[rownumber + 1][:2]

#     if currentT == afterT and currentS == afterS: pass
#     else:
#             if currentT2 == beforeT and currentS == beforeS: pass
#             else: return 2

In [148]:
# create variable to iterate through
shape = fiona.open(XY_Points_outfile)

In [149]:
# for loop to write line file between every 2 sequential points
y = 0
for feat in shape:
    # Create if statement so you y is never > the length of your data set; this will end your script
    if y + 1 < len(filter_dato):
        # What file to write
        file = linesFolder + "\line_" + str(y) + ".shp"
        # Same as previous writer, except QgsWkbTypes. is LineString
        writer = QgsVectorFileWriter(file, 'UTF-8', layerFields, QgsWkbTypes.LineString, \
                                 QgsCoordinateReferenceSystem('EPSG:26919'), 'ESRI Shapefile')
        # y point in point_list will act as the start of the line
        lineStart = point_list[y]
        # y + 1 point in point_list will act as the end of the line
        lineEnd = point_list[y+1]
        # Creates line based on the start and end points previously specified
        line = QgsGeometry.fromPolylineXY([lineStart, lineEnd])
        # Create empty feature for line
        linef = QgsFeature()
        # Set geometry to previously created PolyLine
        linef.setGeometry(line)
        # Append attributes from dataframe
        # Since the Line will consist of 2 points, both points' attributes must be stored in the line
        linef.setAttributes([int(filter_dato['Point Number'].iloc[y]), \
                        filter_dato['Subclass'].iloc[y], filter_dato['Northing'].iloc[y].item(), \
                        filter_dato['Easting'].iloc[y].item(), filter_dato['Altitude'].iloc[y].item(), \
                        filter_dato['Pre Angle'].iloc[y].item(), \
                        filter_dato['Post Angle'].iloc[y].item(), filter_dato['Class'].iloc[y].item(), \
                        filter_dato['mu'].iloc[y].item(), filter_dato['Pre/Post'].iloc[y]])
        # if statement so a line is only written if it belongs to the same transect
        if filter_dato['Subclass'].iloc[y][:2] == filter_dato['Subclass'].iloc[y+1][:2]:
            if filter_dato['Point Number'].iloc[y] + 1 == filter_dato['Point Number'].iloc[y+1]:
                writer.addFeature(linef)
                y += 1
            else: y += 1
        else:
            y += 1
    else: break
del(writer)

C:\Users\brett\anaconda3\envs\CAFE\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: QgsVectorFileWriter constructor is deprecated
  # Remove the CWD from sys.path while we load stuff.


In [150]:
# y = 0
# for feat in shape.iterrows():
#     print(filter_dato['mu'].iloc[y])
#     y += 1

In [151]:
# Create new shapefile for Points that do not create a line

y = 0
shape = gpd.read_file(XY_Points_outfile)

# for loop to write point for every mapping unit
for feat in shape.iterrows():
    # Create if statement so you y is never > the length of your data set; this will end your script
    if filter_dato['mu'].iloc[y] == 1:
            
        # What file to write
        file = mu_Folder + "\mu_" + str(y) + ".shp"
        # Same as previous writer, except QgsWkbTypes. is Polygon
        writer = QgsVectorFileWriter(file, 'UTF-8', layerFields, QgsWkbTypes.MultiPoint, \
                                QgsCoordinateReferenceSystem('EPSG:26919'), 'ESRI Shapefile')
        poly1 = QgsGeometry.fromPointXY(point_list[y])
        feat1 = QgsFeature()
        feat1.setGeometry(poly1)
        feat1.setAttributes([int(filter_dato['Point Number'].iloc[y]), \
                    filter_dato['Subclass'].iloc[y], filter_dato['Northing'].iloc[y].item(), \
                    filter_dato['Easting'].iloc[y].item(), filter_dato['Altitude'].iloc[y].item(), \
                    filter_dato['Pre Angle'].iloc[y].item(), \
                    filter_dato['Post Angle'].iloc[y].item(), filter_dato['Class'].iloc[y].item(), \
                    filter_dato['mu'].iloc[y].item(), filter_dato['Pre/Post'].iloc[y]])
        y += 1
        writer.addFeature(feat1)
    else: 
        y += 1
del(writer)

C:\Users\brett\anaconda3\envs\CAFE\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: QgsVectorFileWriter constructor is deprecated
  from ipykernel import kernelapp as app


Concatenate all PolyLine shapefiles into a single shapefile

In [152]:
# Specify which folder to look in
file = os.listdir(linesFolder)

In [153]:
# Look for all files ending in ".shp"
path = [os.path.join(linesFolder, i) for i in file if ".shp" in i]

# Concatenate to file specified in last line of code
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
# Write to same folder + name of new folder to be created which will include the shapefile of compiled XY Points
# Variable below is the suffix to add to linesFolder to specify folder to store compiled line files
linesCompSuff = "\\Lines_Comp"
gdf.to_file(linesFolder + linesCompSuff)

In [154]:
# Assign compiled lines file path to variable linesCompiled
linesCompiled = (linesFolder + linesCompSuff + linesCompSuff + ".shp")

In [155]:
# # Look for all files ending in ".shp"
# path = [os.path.join(mu_Folder, i) for i in file if ".shp" in i]

# # Concatenate to file specified in last line of code
# gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],
#                         ignore_index=True), crs=gpd.read_file(path[0]).crs)
# # Write to same folder + name of new folder to be created which will include the shapefile of compiled XY Points
# # Variable below is the suffix to add to linesFolder to specify folder to store compiled line files
# muCompSuff = "\\mu_Comp"
# gdf.to_file(mu_Folder + muCompSuff)

In [156]:
# # Assign compiled lines file path to variable linesCompiled
# muCompiled = (mu_Folder + muCompSuff + muCompSuff + ".shp")

Buffer the Lines 

In [157]:
# Buffer lines
# Assign empty folder to variable to write new file to
linesBufferFolder = Buffer_Folder
# Read in line shapefile of transects
lines= gpd.read_file(linesCompiled)
# Assign buffer and keep attributes
lines['geometry'] = lines.buffer(1, cap_style=3)
# Write new shapefile
lines.to_file(linesBufferFolder + "\\Lines_Buffer")

Buffer mapping units

In [158]:
# Specify which folder to look in to buffer mu
file = os.listdir(mu_Folder)

In [159]:
# Assign empty folder to variable to write new file to
muBufferFolder = mu_Buffer_Folder
# # Read in line shapefile of transects
# mus= gpd.read_file(muCompiled)
# # Assign buffer and keep attributes
# mus['geometry'] = mus.buffer(1, cap_style=3).rotate(filter_dato['Post Angle'][1], origin='center')
# # Write new shapefile
# mus.to_file(muBufferFolder + "\\mu_Buffer")

In [160]:
path = [os.path.join(mu_Folder, i) for i in file if ".shp" in i]
y = 0
for i in path:
    mus = gpd.read_file(path[y])
    if mus['Pre/Post'][0] == "post": 
        mus['geometry'] = mus.buffer(1, cap_style=3).rotate(mus["Post Angle"][0], origin='center')
        mus.to_file(muBufferFolder + "\\mu_" + str(mus["PointNum"][0]) + "_Buffer.shp")
    else: 
        mus['geometry'] = mus.buffer(1, cap_style=3).rotate(mus["Pre Angle"][0], origin='center')
        mus.to_file(muBufferFolder + "\\mu_" + str(mus["PointNum"][0]) + "_Buffer.shp")
    y += 1
  


In [161]:
# Specify which folder to look in to compile mu
file = os.listdir(muBufferFolder)

In [162]:
# Look for all files ending in ".shp"
path = [os.path.join(muBufferFolder, i) for i in file if ".shp" in i]

# Concatenate to file specified in last line of code
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
# Write to same folder + name of new folder to be created which will include the shapefile of compiled XY Points
# Variable below is the suffix to add to linesFolder to specify folder to store compiled line files
muCompSuff = "\\mu_Comp"
gdf.to_file(muBufferFolder + muCompSuff)

In [25]:
ditch = waterFilter[28:34]
ditch

,Point Number,Subclass,Northing,Easting,Altitude,Pre Angle,Post Angle,Class,mu,Pre/Post
92,93,21jn,4737063.369,347660.272,1.125,-32.985121,-129.786569,2,0,post:
93,94,21jn,4737058.029,347655.825,1.103,-129.786569,50.651079,2,0,post:
94,95,21jn,4737068.191,347664.157,1.041,50.651079,-58.994755,2,0,post:
95,96,21jn,4737067.785,347664.401,1.048,-58.994755,-127.605635,2,0,post:
96,97,21jn,4737062.331,347660.200,1.037,-127.605635,-127.192770,2,0,post:
97,98,21jn,4737057.044,347656.188,1.046,-127.192770,43.204500,2,0,pre


In [38]:
northList = ditch['Northing'].tolist()
eastList = ditch['Easting'].tolist()

# Swap function must be called because 1st point on ditch polygons is the center point
# Swap allows the polygon to "begin" at a corner; where the last point will connect to
# Swap function 
def swapPositions(list, pos1, pos2): 
      
    list[pos1], list[pos2] = list[pos2], list[pos1] 
    return list
pos1 = 0
pos2 = 1

polygon_geom = Polygon(zip(swapPositions(eastList, pos1, pos2), swapPositions(northList, pos1, pos2)))
crs = {'init': 'epsg:26919'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
print(polygon.geometry)

polygon.to_file(filename = Polygon_Folder + '\ditch1_22jn.shp', driver = 'ESRI Shapefile')

0    POLYGON ((347655.825 4737058.029, 347660.272 4...
Name: geometry, dtype: geometry


In [37]:
pList = ditch['Point Number'].tolist()
pos1 = 0
pos2 = 1
# Swap function 
def swapPositions(list, pos1, pos2): 
      
    list[pos1], list[pos2] = list[pos2], list[pos1] 
    return list
print(swapPositions(pList, pos1, pos2))

[94, 93, 95, 96, 97, 98]


After erasing overlaps in QGIS, concatenate transect buffers and mu buffers together

In [300]:
# Specify files to variables

linebuffFinal = r"G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Red River\Jupyter_Test_RR15aug2019\Lines_Buffered\Lines_Buffer\Lines_Buffer.shp"
muFinal = r"G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Red River\Jupyter_Test_RR15aug2019\mu_Buffered\mu_Comp\mu_Comp.shp"

# Specify output

finalOutput = r"G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\UAS Data Collection\Red River\2019\Transects_Working\RR_15_Aug_2019transects_Final.shp"


In [301]:
# Look for all files ending in ".shp"
path = [linebuffFinal, muFinal]

# Concatenate to file specified in last line of code
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path],
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)
# Write to same folder + name of new folder to be created which will include the shapefile of compiled XY Points
# Variable below is the suffix to add to linesFolder to specify folder to store compiled line files
gdf.to_file(finalOutput)

In [174]:
# final_lineBuffer = r"G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Red River\Jupyter_Test_RR15aug2019\Lines_Buffered\Lines_Buffer\Lines_Buffer.shp"
# final_muBuffer = r"G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\SaltMUAS_share\EPA Salt Marsh UAS Study\Red River\Jupyter_Test_RR15aug2019\mu_Buffered\mu_Comp\mu_Comp.shp"

In [177]:
# # Load shapefiles with fiona

# fc_A = fiona.open(final_lineBuffer)
# fc_B = fiona.open(final_muBuffer)

fiona.collection.Collection

In [183]:
# # List to collect pairs of intersecting features
# fc_intersect = []

# with fiona.open(final_lineBuffer) as fc_A:
#     for featA in fc_A:
#         with fiona.open(final_muBuffer) as fc_B:
#             for featB in fc_B:
#                 if shape(featA['geometry']).intersects(shape(featB['geometry'])):
#                     fc_intersect.append([featA, featB])
# # This creates a list of dictionaries with points of overlap
## https://gis.stackexchange.com/questions/311194/how-to-find-overlapping-polygons-of-two-shapefiles-python

In [ ]:
# for use in the QGIS Python Console to remove buffers from overlapping Transects

layer = iface.activeLayer()
for f1 in layer.getFeatures():
    for f2 in layer.getFeatures():
        if f1.id() < f2.id():
            geom1 = f1.geometry()
            geom2 = f2.geometry()
            new_geom = geom2.difference(geom1)
            layer.dataProvider().changeGeometryValues({f2.id(): new_geom})